In [1]:
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
from experiments.Models import BertBaseline


2024-06-03 21:28:21.414054: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-03 21:28:21.414386: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-03 21:28:21.508968: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-03 21:28:21.705111: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 21:28:22.928262: W tensorflow/compiler/tf2

In [2]:
# model = BertBaseline()
# model.load_model_weights('../models/bert_evidence_baseline_weights.h5')
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
test_dataset = pd.read_csv('../data/emrqa_evidence_test.csv')
tokenized_test = tokenizer(test_dataset['question'].to_list(),
                           test_dataset['evidence'].to_list(),
                           padding='max_length',
                           truncation=True,
                           max_length=512,
                           return_tensors='tf')
test_x = (tokenized_test['input_ids'], tokenized_test['attention_mask'])
test_y = np.stack((np.eye(512)[test_dataset['start_token']], np.eye(512)[test_dataset['end_token']]), axis=1)
results = model.model.predict(test_x, batch_size=32)

2024-06-03 21:28:55.940661: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-03 21:28:56.216452: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-03 21:28:56.216666: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   72         'attention_mask[0][0]']      
                             hidden_state=(None, None,                                        

In [4]:
import tensorflow as tf

In [7]:
model = BertBaseline()
checkpoint = tf.train.Checkpoint(model=model.model)
checkpoint.restore('../models/latest/bert_evidence_model')

Some layers from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model_2 (TFBertMod  TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]',           
 el)                         ngAndCrossAttentions(last_   72         'attention_mask[0][0]']      
                             hidden_state=(None, None,                                      

In [10]:
from get_results import compute_metrics, get_prediction

In [11]:
df = pd.read_csv('../data/emrqa_evidence_test.csv')
tokenized_data = tokenizer(df['question'].to_list(),
                           df['evidence'].to_list(),
                           padding='max_length',
                           truncation=True,
                           max_length=512,
                           return_tensors='tf')
predictions = model.model.predict((tokenized_data['input_ids'], tokenized_data['attention_mask']))
starts = []
ends = []
for i in predictions:
    i = i.T
    starts.append(np.argmax(i[0]))
    ends.append(np.argmax(i[1]))
predictions = pd.DataFrame({"start": starts, "end": ends})
df['predicted_start'], df['predicted_end'] = starts, ends
df['predictions'] = df.apply(
    lambda x: get_prediction(x['evidence'], x['predicted_start'], x['predicted_end']), axis=1)
df['exact_match'], df['precision'], df['recall'], df['f1_score'] = zip(
    *df.apply(lambda x: compute_metrics(x['predictions'], x['answer']), axis=1))
# get average metrics
print(df[['exact_match', 'precision', 'recall', 'f1_score']].mean())

207/207 [==============================] - 96s 462ms/step
exact_match    0.000000
precision      0.143941
recall         0.030434
f1_score       0.045844
dtype: float64


In [31]:
model = BertBaseline()
df = pd.read_csv('../data/emrqa_evidence_test.csv')
tokenized_data = tokenizer(df['question'].to_list(),
                           df['evidence'].to_list(),
                           padding='max_length',
                           truncation=True,
                           max_length=512,
                           return_tensors='tf')
predictions = model.model.predict((tokenized_data['input_ids'], tokenized_data['attention_mask']))
starts = []
ends = []
for i in predictions:
    i = i.T
    starts.append(np.argmax(i[0]))
    ends.append(np.argmax(i[1]))
predictions = pd.DataFrame({"start": starts, "end": ends})
df['predicted_start'], df['predicted_end'] = starts, ends
df['predictions'] = df.apply(
    lambda x: get_prediction(x['evidence'], x['predicted_start'], x['predicted_end']), axis=1)
df['exact_match'], df['precision'], df['recall'], df['f1_score'] = zip(
    *df.apply(lambda x: compute_metrics(x['predictions'], x['answer']), axis=1))
# get average metrics
print(df[['exact_match', 'precision', 'recall', 'f1_score']].mean())

Some layers from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model_4 (TFBertMod  TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]',           
 el)                         ngAndCrossAttentions(last_   72         'attention_mask[0][0]']      
                             hidden_state=(None, None,                                      

In [32]:
model = tf.saved_model.load('../models/latest/bert_evidence_model')
batch_size = 16  # Adjust based on your GPU memory capacity
starts, ends = [], []
for start in range(0, len(df), batch_size):
    end = min(start + batch_size, len(df))
    batch_df = df[start:end]

    # Tokenize batch data
    tokenized_data = tokenizer(
        batch_df['question'].to_list(),
        batch_df['evidence'].to_list(),
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='tf'
    )

    input_data = {
        'input_ids': tokenized_data['input_ids'],
        'attention_mask': tokenized_data['attention_mask'],
    }

    inference_func = model.signatures["serving_default"](**input_data)
    
    for output in inference_func.values():
        predictions = output.numpy()
        # predictions shape: (batch_size, sequence_length, 2)
        start_logits = predictions[:, :, 0]
        end_logits = predictions[:, :, 1]
        starts.extend(np.argmax(start_logits, axis=1))
        ends.extend(np.argmax(end_logits, axis=1))

    # start_logits, end_logits = predictions
    # starts = np.argmax(start_logits, axis=-1)
    # ends = np.argmax(end_logits, axis=-1)

print(len(starts), len(ends))
df['predicted_start'] = starts
df['predicted_end'] = ends
df['predictions'] = df.apply(
    lambda x: get_prediction(x['evidence'], x['predicted_start'], x['predicted_end']), axis=1)
df['exact_match'], df['precision'], df['recall'], df['f1_score'] = zip(
    *df.apply(lambda x: compute_metrics(x['predictions'], x['answer']), axis=1))
# get average metrics
print(df[['exact_match', 'precision', 'recall', 'f1_score']].mean())

6601 6601
exact_match    0.000303
precision      0.026768
recall         0.021235
f1_score       0.021075
dtype: float64


In [3]:
starts = []
ends = []
for i in results:
    i = i.T
    starts.append(np.argmax(i[0]))
    ends.append(np.argmax(i[1]))
predictions = pd.DataFrame({"start": starts, "end": ends})
predictions

,start,end
0,218,110
1,23,130
2,33,111
3,323,443
4,6,414
...,...,...
6596,17,440
6597,511,143
6598,447,425
6599,21,237


In [5]:
starts_truth = []
ends_truth = []
for i in test_y:
    starts_truth.append(np.argmax(i[0]))
    ends_truth.append(np.argmax(i[1]))
truth = pd.DataFrame({"start": starts_truth, "end": ends_truth})
truth

,start,end
0,25,27
1,40,40
2,20,23
3,33,38
4,10,13
...,...,...
6596,30,32
6597,15,16
6598,12,24
6599,22,27


In [6]:
incorrect = 0
for ind, i in enumerate(truth):
    t = truth.iloc[ind].to_numpy()
    p = predictions.iloc[ind].to_numpy()
    if not (sum(t == p) == 2):
        incorrect += 1
print(incorrect)

0


In [6]:
sum(t == p)

2